# Lectura de Datos

In [1]:
# Importar librerías
import pandas
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [8]:
# Lectura de los datos
data = pandas.read_csv("./IMDb_movies/IMDb movies.csv")


C:\Users\Alex\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Limpieza y nuevas variables

In [9]:
# Limpieza de Género. Hace dummies para cada género distinto. 
generos = []
for j in data['genre'].dropna().str.rsplit(', '):
    for i in j:
        if i not in generos:
            generos.append(i)

generos_names = ["{}{}".format(i,"_Genre") for i in generos]

aux = []
for i in range(len(generos)):
    aux.append(pandas.Series.tolist(data['genre'].str.contains(generos[i], regex = False)))

Genres = pandas.DataFrame.transpose(pandas.DataFrame(aux, index = generos))

Genres.columns = generos_names

In [10]:
# Limpieza de Lenguaje. Hace dummies para cada lenguaje distinto. 
# Faltaría hacer una categoría de otros para reducir la cantidad de variables.
lenguaje = []
for j in data['language'].dropna().str.rsplit(', '):
    for i in j:
        if i not in lenguaje:
            lenguaje.append(i)

lenguaje_names = ["{}{}".format(i,"_Language") for i in lenguaje]

aux = []
for i in range(len(lenguaje)):
    aux.append(pandas.Series.tolist(data['language'].str.contains(lenguaje[i], regex = False)))

Lenguajes = pandas.DataFrame.transpose(pandas.DataFrame(aux, index = lenguaje))
Lenguajes.columns = lenguaje_names

In [11]:
# Limpieza de País. Hace dummies para cada pais distinto. 
# Faltaría hacer una categoría de otros para reducir la cantidad de variables.
pais = []
for j in data['country'].dropna().str.rsplit(', '):
    for i in j:
        if i not in pais:
            pais.append(i)

pais_names = ["{}{}".format(i,"_Country") for i in pais]

aux = []
for i in range(len(pais)):
    aux.append(pandas.Series.tolist(data['country'].str.contains(pais[i], regex = False)))

Paises = pandas.DataFrame.transpose(pandas.DataFrame(aux, index = pais))
Paises.columns = pais_names

In [12]:
# Unir los dataframes, para que incluya todas las de generos distintos (los dummies)
data = pandas.concat([data, Genres], axis=1)
data = pandas.concat([data, Lenguajes], axis=1)
data = pandas.concat([data, Paises], axis=1)
# Quitamos todas las observaciones con NAs
data = pandas.DataFrame.dropna(data)

In [13]:
# Categóricas
# Año
data["year"] = data["year"].astype('category').cat.codes

# Production Company
data["production_company"] = data["production_company"].astype('category').cat.codes
data.loc[data['production_company'].isin((data['production_company'].value_counts() \
                                          [data['production_company'].value_counts() < 35]).index), \
                                          'production_company'] = 0

# Director
data["director"] = data["director"].astype('category').cat.codes
data.loc[data['director'].isin((data['director'].value_counts() \
                                [data['director'].value_counts() < 5]).index), 'director'] = 0

# Writer
data["writer"] = data["writer"].astype('category').cat.codes
data.loc[data['writer'].isin((data['writer'].value_counts() \
                                [data['writer'].value_counts() < 5]).index), 'writer'] = 0

In [14]:
data = pandas.get_dummies(data, columns=['director',"writer","production_company"])

In [15]:
# USA Gross Income a log de eso limpio
usa_income_series = pandas.Series.dropna(data['usa_gross_income'])
usa_income_series = usa_income_series[usa_income_series.astype('str').str.contains("$", regex = False)]
usa_income_series = pandas.to_numeric(pandas.Series(usa_income_series.astype(str).str.extract('(\d+)')[0]))
data["usa_gross_income"] = np.log(usa_income_series)

In [16]:
# Worldwide Gross Income a log de eso limpio
worldwide_income_series = pandas.Series.dropna(data['worlwide_gross_income'])
worldwide_income_series = worldwide_income_series[worldwide_income_series.astype('str').str.contains("$", regex = False)]
worldwide_income_series = pandas.to_numeric(pandas.Series(worldwide_income_series.astype(str).str.extract('(\d+)')[0]))
data["worlwide_gross_income"] = np.log(worldwide_income_series)

In [17]:
# Budget a log, pero limpio
budget_series = pandas.Series.dropna(data['budget'])
budget_series = budget_series[budget_series.astype('str').str.contains("$", regex = False)]
budget_series = pandas.to_numeric(pandas.Series(budget_series.astype(str).str.extract('(\d+)')[0]))
data["budget"] = budget_series
data["budget"] = data["budget"].fillna(0)

In [18]:
data["gross_income"] = data["worlwide_gross_income"] + data["usa_gross_income"]

# Fecha a Categóricas

In [19]:
fecha = pandas.to_datetime(data["date_published"])
data["mes"] = fecha.dt.month
data["dia_semana"] = fecha.dt.weekday

data = pandas.get_dummies(data, columns=['mes',"dia_semana"])

# Categoricas a dummies

In [20]:
director_list = [s for s in list(data.columns) if "director" in s]
writer_list = [s for s in list(data.columns) if "writer" in s]
prodcomp_list = [s for s in list(data.columns) if "production" in s]

# Escribiendo a archivo nuevo

In [21]:
data.to_csv("IMDb_Limpio.csv", encoding = "utf-8")

In [22]:
2+2

4